In [184]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import SearchIO
import csv
import sqlite3
import time
from multiprocessing import Pool, cpu_count
import sys
from calc_icity_ import *
import ast
from collections import defaultdict
import subprocess
import numpy as np
import sys

In [191]:
def get_prot_sequence(pid):
    con = sqlite3.connect("80kprotein_stats.db")
    cur = con.cursor()
    cmd = "SELECT sequence FROM proteins WHERE pid = '%s'" % pid 
    #print(cmd)
    cur.execute(cmd)
    return str(cur.fetchone()[0])
    con.close()
def get_faas_protidlist(protidlist, outfile_path, minlen):
    with open(outfile_path, "w") as outfile:
        for protid in protidlist:
            protseq = get_prot_sequence(protid)
            if len(protseq) > minlen:
                print(">" + protid, file=outfile)
                print(protseq, file=outfile)
def get_headers_faa(inpath):
    pids = []
    with open(inpath) as handle:
        for rec in SeqIO.parse(handle, 'fasta'):
            pid = rec.id
            pids.append(pid)
    return pids

## alignment

- always start with all p60 tnpBs id.txt
- add 100 high icity dtnpBs to "ignore" set
- add 100 low icity dtnpBs to "ignore" set
- select 300 random p60 tnpB ids
- p60 cluster again

### get protein ids we want to cluster: tnpB, dtnpB low icity, dtnpB high icity

In [206]:
# # concatenate catalytically active tnpBs to inactive list + condense for visibility
tnpB_p60s_path = "../tnpBs/cluster/OUTPUT/tmp/clu_cluster.tsv"
tnpB_p60_df = pd.read_csv(tnpB_p60s_path, sep = '\t', header = None).rename(columns = {0:"p60",1:'p100'}).drop_duplicates()
tnpB_p60_set = set(tnpB_p60_df["p60"])

tnpB_df = pd.read_csv("tnpB_icity_output.tsv", sep='\t')
tnpB_df_low_icity = tnpB_df[tnpB_df["denom"] > 10][tnpB_df["icity"] < .7].sort_values(["icity","numer"], ascending = False).drop_duplicates()
tnpB_df = tnpB_df[tnpB_df["icity"] > .7].sort_values(["icity","numer"], ascending = False).drop_duplicates()
tnpB_10_target_annot_df = tnpB_df[tnpB_df["denom"] > 10]

high_icity_dtnpBs_ls = []
for baitp100s_str in tnpB_10_target_annot_df["baitp100s"]:
    baitp100s_ls = ast.literal_eval(baitp100s_str)
    high_icity_dtnpBs_ls += baitp100s_ls
high_icity_dtnpBs_ls = list(set(high_icity_dtnpBs_ls))
high_icity_dtnpBs_samp = set(np.random.choice(high_icity_dtnpBs_ls, 100))

low_icity_dtnpBs_ls = []
for baitp100s_str in tnpB_df_low_icity["baitp100s"]:
    baitp100s_ls = ast.literal_eval(baitp100s_str)
    low_icity_dtnpBs_ls += baitp100s_ls
low_icity_dtnpBs_ls = list(set(low_icity_dtnpBs_ls))
low_icity_dtnpBs_samp = set(np.random.choice(low_icity_dtnpBs_ls, 100))

tnpBs_p60samp = [tnpB for tnpB in np.random.choice(
    list(set(tnpB_p60_df["p60"])), 300, replace = False) 
                 if (tnpB not in low_icity_dtnpBs_samp) 
                 and (tnpB not in high_icity_dtnpBs_samp)]


/tmp/ipykernel_29511/3351818201.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tnpB_df_low_icity = tnpB_df[tnpB_df["denom"] > 10][tnpB_df["icity"] < .7].sort_values(["icity","numer"], ascending = False).drop_duplicates()


### cluster

In [207]:
def cluster_(pid_inpath, mmseqs_dir, identity, pid_list = None):
    if pid_list != None:
        with open(pid_inpath, "w") as outfile:
            for pid in pid_list:
                print(pid, file=outfile)
    cmd = "python3 clustering_gen.py {} {} {}".format(pid_inpath, mmseqs_dir, identity)
    process = subprocess.Popen(cmd.split(' '))
    process.wait()
    output_tsv = mmseqs_dir + "/OUTPUT/_cluster.tsv"
    cluster_rep_df = pd.read_csv(output_tsv, sep = '\t')
    return list(set(cluster_rep_df.iloc[:,0]))

In [ ]:
# inactive_tnpBs_list = []
# with open("../inactive_tnpbs_ref.3.faa", "r") as infile:
#     lines=infile.readlines()
#     for line in lines:
#         if line[0] == '>':
#             tnpBid = line.strip('>').strip('\n')
#             inactive_tnpBs_list.append(tnpBid)

In [ ]:
# pid_inpath = "../tnpBs/_all_inactive_tnpBs.3.txt"
# mmseqs_dir = "../tnpBs/clust_inactive"
# identity = "0.6"
# pid_list = inactive_tnpBs_list

In [ ]:
# dTnpB_p60_list = cluster_(pid_inpath, mmseqs_dir, identity, pid_list = None)

In [204]:
# len(pid_list), len(dTnpB_p60_list)

In [218]:
len(low_icity_dtnpBs_samp), len(tnpBs_samp_p60_list)

(100, 384)

In [210]:
pid_inpath = "../tnpBs/tnpBs_dtnpBs_samp.3.txt"
mmseqs_dir = "../tnpBs/clust_inactive_active"
identity = "0.6"
tnpBs_samp_set = high_icity_dtnpBs_samp.copy()
tnpBs_samp_set.update(low_icity_dtnpBs_samp)
tnpBs_samp_set.update(tnpBs_p60samp)

#tnpBs_samp_p60_list = cluster_(pid_inpath, mmseqs_dir, identity, list(tnpBs_samp_set))

In [213]:
len(tnpBs_samp_set), len(tnpBs_samp_p60_list)

(493, 384)

### get alignment infile

In [219]:
protidlist = tnpBs_samp_p60_list
kalign_infile = "../tnpBs/alignments/kalign_tnpB_dTnpB_p60_ref.in.faa"
get_faas_protidlist(protidlist, kalign_infile, minlen = 300)

### kalign

In [220]:
infile_k = kalign_infile
outfile_k = "../tnpBs/alignments/kalign_tnpB_dTnpB_p60_ref.out.faa"

In [221]:
def kalign(infile, outfile, minlen = 300, idlist = None):
    if idlist != None:
        with open(infile, "w") as kalign_infile:
            for protid in idlist:
                protseq = get_prot_sequence(protid)
                if len(protseq) > minlen:
                    print(">" + protid, file=kalign_infile)
                    print(protseq, file=kalign_infile)
    cmd = "kalign -i {} -o {}".format(infile, outfile)
    process = subprocess.Popen(cmd.split(' '))

In [ ]:
kalign(infile_k, outfile_k)

## Build tree

In [222]:
infile_tree = outfile_k
outfile_tree = outfile_k.replace("faa", "tree").replace("kalign_", "")

In [223]:
def fasttree(infile, outfile):
    cmd = "FastTree {} > {}".format(infile, outfile)
    print(cmd)
    process = subprocess.Popen(cmd.split(' '))

In [225]:
# fasttree(infile_tree, outfile_tree)

### create dataframe ann.otating tnpBs on tree

In [226]:
tnpB_ids_tree = get_headers_faa(infile_tree)
tnpB_isinactive_highicity_tree = [1 if tnpB in high_icity_dtnpBs_samp else 0 for tnpB in tnpB_ids_tree ]
tnpB_isinactive_lowicity_tree = [1 if tnpB in low_icity_dtnpBs_samp else 0 for tnpB in tnpB_ids_tree ]
tnpB_isactive_tree = [1 if tnpB in tnpBs_p60samp else 0 for tnpB in tnpB_ids_tree ]

In [240]:
tnpB_treelabels_df = pd.DataFrame(data={"tnpB_id": tnpB_ids_tree, 
                   "inactive_highicity": tnpB_isinactive_highicity_tree,
                   "inactive_lowicity": tnpB_isinactive_lowicity_tree,
                   "active": tnpB_isactive_tree
                  })
tnpB_treelabels_df['activity'] = tnpB_treelabels_df.iloc[:,1:].idxmax(1)
tnpB_treelabels_df.to_csv("../tnpBs/alignments/tnpB_dTnpB_p60_treelabels.csv")

### Extracting info from trees

In [242]:
sum(tnpB_isinactive_highicity_tree)

18

In [ ]:
# Clusters of inactive high-icity tnpBs, starting at 12:00
